In [1]:
# Import packages
import os
import csv
import json
import codecs

In [2]:
# Retrieve Files from dataset
headers = ['File Name','Competition ID','Kernel Created Date', 'Votes']

abs_path = '/home/ictstudent/ButterChicken/Data Preparation/'
notebook_lists_path = 'Notebook Lists'
os.chdir(abs_path + notebook_lists_path)

comp_id_mapping = {}

retrieve_files = {}
notebook_lists = os.listdir('.')

# Chapter 3: Evaluation
avoid_list = [
    "gpreda_santander-eda-and-prediction.ipynb",
    "artgor_eda-and-models.ipynb",
]

for notebook_list in notebook_lists:
    if notebook_list[-4:] == '.csv':
        file_count = 0
        with open(notebook_list, 'r') as file:
            data = csv.reader(file, delimiter=',')
            comp_id = next(data)[1]
            comp_id_mapping[notebook_list] = comp_id
        # end with
    # end if
# end for

for notebook_list in notebook_lists:
    if notebook_list[-4:] == '.csv':
        file_count = 0
        with open(notebook_list, 'r') as file:
            data = csv.reader(file, delimiter=',')
            comp_id = comp_id_mapping[notebook_list]
            comp_id_path = '../Competitions/' + comp_id
            
            if os.path.exists(comp_id_path):
                os.system('rm -rf ' + comp_id_path)
            # end if
            
            if not os.path.exists(comp_id_path):
                os.system('mkdir ' + comp_id_path)
            # end if
            
            for row in data:
                if row[0] in avoid_list: continue
                notebook_path = '../../../KGTorrent/KT_dataset/' + row[0]
                command = 'cp ' + notebook_path + ' ' + comp_id_path
                if os.path.exists(notebook_path):
                    os.system(command)
                    file_count = file_count + 1
                # end if
            # end for
            retrieve_files[comp_id] = file_count
        # end with
    # end if
# end for

total_files = 0

for key in retrieve_files:
    each_comp_files = retrieve_files[key]
    total_files = total_files + each_comp_files
    print(str(each_comp_files) + '/100 files are copied from Competition ' + str(key) + '.');
#end for

77/100 files are copied from Competition 10385.
92/100 files are copied from Competition 14242.
65/100 files are copied from Competition 9120.


In [3]:
def extract_notebook(index, file_path):
    code = ''
    found_code = False
    markdown_list = []
    md_code_pair = []
    
    file = codecs.open(file_path, 'r')
    source = file.read()
    notebook = json.loads(source)

    for row in notebook['cells']:
        markdown = ''
                
        if row['cell_type'] == 'markdown':
            if found_code:
                md_code_pair.append({'markdown': markdown_list , 'code': code})
                code = ''
                markdown_list = []
                index = index + 1
            # end if
            
            for line in row['source']:            
                markdown = markdown + line
            # end for
            found_code = False
            markdown_list.append(markdown)    
        # end if
            
        if row['cell_type'] == 'code':
            if found_code:
                code = code + '\n'
            # end if

            for line in row['source']:
                code = code + line
            # end for
            found_code = True
        # end if
        
    # end for
        
    return index, md_code_pair
# end def

def append_path(cur, dest):
    return cur + '/' + dest
# end def

In [4]:
all_pairs = []
cur_index = 0

non_code_notebooks = []

path = abs_path + 'Competitions'
os.chdir(path)
competitions = os.listdir('.')

for competition in competitions:
    if os.path.isdir(competition):
        notebooks = os.listdir(competition)
        comp_path = append_path(path, competition)
        
        for notebook in notebooks:
            if notebook[-6:] == '.ipynb':
                notebook_path = append_path(comp_path, notebook)
                with open(notebook_path, 'r') as file:
                    cur_index, output = extract_notebook(cur_index, notebook_path)
                    if output == []:
                        non_code_notebooks.append(notebook_path)
                    else:
                        for pair in output:
                            all_pairs.append(pair)
                        # end for
                    # end if
                # end with
        # end for
    # end if
# end for

In [5]:
json_file_name = '../nl_pl_pairs_chap3.json'

with open(json_file_name, 'w+') as output_file:
    json.dump(all_pairs, output_file)
# end with

In [6]:
non_code_files = len(non_code_notebooks)

print('There are ' + str(non_code_files) + ' notebooks which are not contain any code cell.')
print('The output JSON file is created from ' + 
      str(total_files - non_code_files)+ '/' + str(total_files) + ' notebooks.')

There are 49 notebooks which are not contain any code cell.
The output JSON file is created from 185/234 notebooks.
